# MongoDB Atlas Full-Text Search Retrieval with Claude

This notebook provides a step-by-step guide for using the MongoDB Atlas search tool with Claude. We will:

1. Set up the environment and imports
2. Build a search tool to query an MongoDB Atlas cluster
3. Test the search tool  
4. Create a Claude client with access to the tool 
5. Compare Claude's responses with and without access to the tool

## Imports and Configuration 

First we'll import libraries and load environment variables. This includes setting up logging so we can monitor the process.

In [13]:
import os
import sys
import dotenv
import anthropic

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import claude_retriever

# Load environment variables
dotenv.load_dotenv(dotenv_path="../.env.local")

True

In [14]:
# Import and configure logging 
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to log to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

In [15]:
mongo_connection_string = os.environ['MONGO_CONNECTION_STR']
mong_db_name = os.environ['MONG_DB_NAME']
mongo_collection_name = os.environ['MONG_COLLECTION_NAME']

# Store your data

The first step is setting up your datastore. Here, we will make use of the [Kaggle Amazon Products 2020 Dataset](https://www.kaggle.com/datasets/promptcloud/amazon-product-dataset-2020). It contains 10000 products from Amazon, including their product title, description, price, category tags, etc. For the purposes of this notebook, we've pre-processed the data to concatenate the title, description and category tags into a single "document" field and saved it locally as a JSONL with one line for each product.

We now need to transform this raw text dataset into an embedding dataset. In this notebook we will opt for the simplest possible way to do this locally:

1. We will use the [sentence-transformers](https://www.sbert.net/index.html) library, which allows us to use a lightweight model to embed our text data using only a CPU if that is all we have available.
2. We will save the text/embedding pairs on MongoDB Atlas that can be used to query and augment the custom prompts to LLM.



# Create a Search Tool for your data

We now create a Search Tool, which can add data, take queries and return formatted relevant results. We also need to describe what the search tool will return, which Claude will read to make sure it is correctly used.

In [91]:
from typing import Optional
from dataclasses import dataclass
from abc import ABC, abstractmethod

from anthropic import Anthropic
from pymongo import MongoClient
import pandas as pd

import logging
logger = logging.getLogger(__name__)

@dataclass
class SearchResult:
    """
    A single search result.
    """
    content: str

class Tool(ABC):
    tool_description: str

# MongoDB Atlas Full-Text Searcher

class MongoDBAtlasSearchTool(object):

    _index_mapping = {"mappings":{"dynamic":True}}
    _index_name = "claude_search_index"

    def __init__(self,
                tool_description: str,
                mongo_connection_string: str,
                mongo_database: str,
                mongo_collection: str,
                project_fields:Optional[str] = {"_id":0, "score": { "$meta": "searchScore" }},
                output_field:Optional[str] = "text",
                query_field:Optional[str] = "text",
                truncate_to_n_tokens: Optional[int] = 5000):
        
        self.connection_string = mongo_connection_string
        self.database_name = mongo_database
        self.collection_name = mongo_collection
        self.query_project_fields = project_fields
        self.query_field = query_field
        self.output_field = output_field
        self._connect_to_mongodb_atlas()

        self.tool_description = tool_description
        self.truncate_to_n_tokens = truncate_to_n_tokens
        if truncate_to_n_tokens is not None:
            self.tokenizer = Anthropic().get_tokenizer() 
    
    def _connect_to_mongodb_atlas(self):
        self._client = MongoClient(self.connection_string)
        self._collection = self._client[self.database_name][self.collection_name]
        if not self._collection.find_one():
            raise ValueError(f"MongoDB collection {self.database_name}.{self.collection_name} does not exist.")
        
    def _check_index(self):
        indexes = list(self._collection.list_search_indexes())
        idx = list(filter(lambda x: x["name"] == self._index_name, indexes))
        if len(list(idx)) > 0:
            logging.warn(f"MongoDB collection {self.database_name}.{self.collection_name} already has a search index. Dropping it.")
            self._collection.drop_index("claude_search_index")
        return False
        
    def _create_search_index(self):
        definition = {"definition":self._index_mapping, "name": self._index_name}
        if not self._check_index():
            self._collection.create_search_index(definition)

    def upload_data_to_mongodb(self, fileName: str):
        try:
            file_extension = fileName.split(".")[-1]
            print(f"File Extension:{file_extension}")
            if file_extension.endswith("csv"):
                df = pd.read_csv(fileName)
            elif file_extension.endswith("jsonl"):
                df = pd.read_json(fileName, orient="records", lines=True)
            if not df.shape[0] > 0:
                raise ValueError(f"Failed to parse file {fileName}")
            else:
                self._collection.insert_many(df.to_dict(orient="records"))
                self._create_search_index()
        except:
            raise ValueError(f"Failed to read file using pandas")
        
    def _format_results(self,extracted: list[str]) -> str:
            """
            Joins and formats the extracted search results as a string.

            :param extracted: The extracted search results to format.
            """
            result = "\n".join(
                [
                    f'<item index="{i+1}">\n<page_content>\n{r}\n</page_content>\n</item>'
                    for i, r in enumerate(extracted)
                ]
            )
            return result
        
    def _format_results_full(self,extracted: list[str]) -> str:
        """
        Formats the extracted search results as a string, including the <search_results> tags.

        :param extracted: The extracted search results to format.
        """
        return f"\n<search_results>\n{self._format_results(extracted)}\n</search_results>"
        
    def truncate_page_content(self, page_content: str) -> str:
        if self.truncate_to_n_tokens is None:
            return page_content.strip()
        else:
            return self.tokenizer.decode(self.tokenizer.encode(page_content).ids[:self.truncate_to_n_tokens]).strip()

    def raw_search(self, query: str, n_search_results_to_use=100) -> list[SearchResult]:
        pipeline = [{"$search": {"index": self._index_name, "text": {"path": self.query_field, "query": query, "fuzzy": {}}}},{"$project": self.query_project_fields}, {"$limit": n_search_results_to_use}]
        results = self._collection.aggregate(pipeline)
        search_results: list[SearchResult] = []
        for result in results:
            search_results.append(SearchResult(content=result[self.output_field]))
        return search_results

    
    def process_raw_search_results(self, results: list[SearchResult]) -> list[str]:
        processed_search_results = [self.truncate_page_content(result.content) for result in results]
        return processed_search_results
    
    def search(self, query: str, n_search_results_to_use: int) -> str:
        raw_search_results = self.raw_search(query, n_search_results_to_use)
        processed_search_results = self.process_raw_search_results(raw_search_results)
        displayable_search_results = self._format_results_full(processed_search_results)
        return displayable_search_results

In [92]:
# Set up MongoDB Atlas instance and upload the data
AMAZON_SEARCH_TOOL_DESCRIPTION = 'The search engine will search over the Amazon Product database, and return for each product its title, description, and a set of tags.'
amazon_search_tool = MongoDBAtlasSearchTool(
    tool_description=AMAZON_SEARCH_TOOL_DESCRIPTION,
    mongo_connection_string= mongo_connection_string,
    mongo_database= mong_db_name,
    mongo_collection= mongo_collection_name,
    query_field= "text",
    project_fields= {"_id":0 , "score": {"$meta": "searchScore"}, "text": 1},
    output_field= "text",
)

In [58]:
# Load data to mongodb [Optional]
# this will add data to the mongodb collection specified and also create a dynamic index
# for more customized index, `amazon_search_tool.index_mapping` can be updated with custom mapping
amazon_search_tool.upload_data_to_mongodb("./data/amazon-products.jsonl")

File Extension:jsonl


Let's test it to see if the tool works!

In [93]:
dinos = amazon_search_tool.search("fun kids dinosaur book", n_search_results_to_use=3)
print(dinos)


<search_results>
<item index="1">
<page_content>
Product Name: Wild Republic T-Rex Plush, Dinosaur Stuffed Animal, Plush Toy, Gifts for Kids, Dinosauria 17 Inches

About Product: Bring dinosaurs to life for your child with this realistic T Rex toy that is designed to capture the detail of this King of the dinosaurs | Wild Republic Stuffed dinosaur toys are a great way for kids to play and learn at the same time | Collect each dino in our 17" Dinosaurian collection which includes the Stegosaurus, Triceratops, Velociraptor, Diplodocus, and the T Rex | Your child will enjoy this stuffed dinosaur Plush because of its realistic design and pose. The textured fabric truly brings this T-Rex to life | Great birthday gift for boys, dinosaur themed birthday parties or bedrooms. Tyrannosaurus Rex is the most popular of the stuffed dinosaurs

Categories: Toys & Games | Stuffed Animals & Plush Toys | Stuffed Animals & Teddy Bears
</page_content>
</item>
<item index="2">
<page_content>
Product Name:

# Use Claude with Retrieval

We can now simply pass this search tool to Claude to use, much in the same way a person might.

In [94]:
ANTHROPIC_SEARCH_MODEL = "claude-2"

client = claude_retriever.ClientWithRetrieval(api_key=os.environ['ANTHROPIC_API_KEY'], search_tool = amazon_search_tool)

query = "I want to get my daughter more interested in science. What kind of gifts should I get her?"
prompt = f'{anthropic.HUMAN_PROMPT} {query}{anthropic.AI_PROMPT}'

Here is the basic response to the query (no access to the tool).

In [95]:
basic_response = client.completions.create(
    prompt=prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)
print('-'*50)
print('Basic response:')
print(prompt + basic_response.completion)
print('-'*50)

2024-03-14 14:35:48,205 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2024-03-14 14:35:48,205 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
--------------------------------------------------
Basic response:


Human: I want to get my daughter more interested in science. What kind of gifts should I get her?

Assistant: Here are some science-related gift ideas to spark your daughter's interest:

- Microscope or telescope - Let her explore tiny worlds or gaze at stars and planets. Get one suited for beginners.

- Chemistry set - Look for a basic set that allows her to do safe, kid-friendly experiments. Supervise to start.

- Robot building kit - She can construct then program motorized creations, teaching coding too. Go for her age level. 

- Nature field guides - Cater to her interests whether it's rocks, bugs, birds, plants, or more. Reference and ID books to use outdoors.

- Magnifying glass - A 

Now we get the same completion, but give Claude the ability to use the tool when thinking about the response.

In [96]:
augmented_response = client.completion_with_retrieval(
    query=query,
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=3,
    max_tokens_to_sample=1000)

print('-'*50)
print('Augmented response:')
print(prompt + augmented_response)
print('-'*50)

2024-03-14 14:36:02,954 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2024-03-14 14:36:02,954 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2024-03-14 14:36:02,957 - claude_retriever.client - INFO -  <thinking>
To gather relevant information to help answer this query, I should search for science kits, toys, or books that are designed specifically to spark interest and engagement with science in young girls. Relevant factors to consider include the age/interests of the daughter, the scientific field or concept the gift focuses on, the learning format (e.g. hands-on kits vs books), and reviews indicating the gift sparked curiosity and engagement.
</thinking>

<search_query>science gifts for girls age 10
2024-03-14 14:36:02,957 - claude_retriever.client - INFO -  <thinking>
To gather relevant information to help answer this query, I should search for science kits, toys, or books that are designed s

Often, you'll want finer-grained control about how exactly Claude uses the results. For this workflow we recommend "retrieve then complete".

In [97]:
relevant_search_results = client.retrieve(
    query=query,
    stop_sequences=[anthropic.HUMAN_PROMPT, 'END_OF_SEARCH'],
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=3,
    max_searches_to_try=5,
    max_tokens_to_sample=1000)

print('-'*50)
print('Relevant results:')
print(relevant_search_results)
print('-'*50)

2024-03-14 14:36:55,393 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2024-03-14 14:36:55,393 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2024-03-14 14:36:55,395 - claude_retriever.client - INFO -  <thinking>
To gather relevant information to help answer this query, I should search for science gifts and activities targeted at girls in my daughter's age range. Information about the appropriateness of different science gifts for different ages as well as reviews on the educational value and engaging qualities of different science toys and kits would also be useful.
</thinking>

<search_query>science gifts for 12 year old girls
2024-03-14 14:36:55,395 - claude_retriever.client - INFO -  <thinking>
To gather relevant information to help answer this query, I should search for science gifts and activities targeted at girls in my daughter's age range. Information about the appropriateness of differen

In [98]:
qa_prompt = f'''{anthropic.HUMAN_PROMPT} You are a friendly product recommender. Here is a query issued by a user looking for product recommendations:

{query}

Here are a set of search results that might be helpful for answering the user's query:

{relevant_search_results}

Once again, here is the user's query:

<query>{query}</query>

Please write a response to the user that answers their query and provides them with helpful product recommendations. Feel free to use the search results above to help you write your response, or ignore them if they are not helpful.

At the end of your response, under "Products you might like:", list the top 3 product names from the search results that you think the user would most like.

Please ensure your results are in the following format:

<result>
Your response to the user's query.
</result>
<recommendations>
Products you might like:
1. Product name
2. Product name
3. Product name
</recommendations>{anthropic.AI_PROMPT}'''

response = client.completions.create(
    prompt=qa_prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)

print('-'*50)
print('Response:')
print(response.completion)
print('-'*50)

2024-03-14 14:37:32,349 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2024-03-14 14:37:32,349 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
--------------------------------------------------
Response:
 <result>
Based on your goal of getting your daughter more interested in science, I would recommend getting her an engaging science kit or toy that allows her to conduct experiments and explore scientific concepts. The "Scientific Explorer My First Science Experiment Kit" seems like an excellent option that would spark her creativity and curiosity through hands-on activities like growing crystals, mixing colors, and more. This represents STEM principles that are great for kids to learn.

Some other ideas would be an astronomy kit with a beginner telescope so she can gaze at planets and stars, a robotics/coding toy she can program, or a kids chemistry set to do safe experiments. Try to find kits tha